In [4]:

import math, os, sys, time, json, random
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from typing import Tuple
import pandas as pd

# Reproducibility
def set_seed(s: int = 42):
    random.seed(s); np.random.seed(s); torch.manual_seed(s)

dfr = pd.read_csv('data/raw/sav_0927_v2.csv')
dfr = dfr[dfr['REPORT_TYPE'] == 'FM-15']
dfr.head(21)

/var/folders/21/m8s3nsm95zd7kmjs3hkqr2pr0000gn/T/ipykernel_74757/2115957890.py:14: DtypeWarning: Columns (3,29,30,34,48,49,50,51,55,64,65,70,71,125,171,172,178,182,190,213,214,215,219) have mixed types. Specify dtype option on import or set low_memory=False.
  dfr = pd.read_csv('data/raw/sav_0927_v2.csv')


,STATION,NAME,DATE,SOURCE,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,AA1,AA2,AA3,...,TMP,UA1,UG1,UG2,VIS,WA1,WD1,WG1,WJ1,WND
3,72207003822,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T00:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,"+0233,5",NaN,NaN,NaN,"016093,5,N,5",NaN,NaN,NaN,NaN,"030,5,N,0041,5"
5,72207003822,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T01:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,"+0228,5",NaN,NaN,NaN,"016093,5,N,5",NaN,NaN,NaN,NaN,"020,5,N,0031,5"
8,72207003822,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T02:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,"+0222,5",NaN,NaN,NaN,"016093,5,N,5",NaN,NaN,NaN,NaN,"030,5,N,0036,5"
10,72207003822,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T03:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,"+0222,5",NaN,NaN,NaN,"016093,5,N,5",NaN,NaN,NaN,NaN,"020,5,N,0036,5"
11,72207003822,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T04:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,"+0222,5",NaN,NaN,NaN,"016093,5,N,5",NaN,NaN,NaN,NaN,"020,5,N,0031,5"
16,72207003822,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T05:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,"+0222,5",NaN,NaN,NaN,"016093,5,N,5",NaN,NaN,NaN,NaN,"360,5,N,0031,5"
19,72207003822,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T06:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,"+0222,5",NaN,NaN,NaN,"016093,5,N,5",NaN,NaN,NaN,NaN,"020,5,N,0036,5"
21,72207003822,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T07:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,"+0217,5",NaN,NaN,NaN,"016093,5,N,5",NaN,NaN,NaN,NaN,"030,5,N,0036,5"
24,72207003822,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T08:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,"+0217,5",NaN,NaN,NaN,"016093,5,N,5",NaN,NaN,NaN,NaN,"020,5,N,0031,5"
27,72207003822,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T09:53:00,7,FM-15,KSAV,V030,"01,0000,2,5",NaN,NaN,...,"+0211,5",NaN,NaN,NaN,"009656,5,N,5",NaN,NaN,NaN,NaN,"020,5,N,0041,5"


In [10]:
import numpy as np
import pandas as pd

def dt_treatment(df, date_col='DATE', expected_minute=53, tolerance_min=15):
    df = (df.copy()
            .assign(DATE_raw=lambda x: x[date_col],
                    _dt=lambda x: pd.to_datetime(x[date_col], errors='coerce')))
    floor = df['_dt'].dt.floor('h')
    dev   = (df['_dt'] - floor - pd.Timedelta(minutes=expected_minute)).abs()
    tol   = pd.Timedelta(minutes=tolerance_min)

    df = df.assign(
        within_tol=dev <= tol,
        dt_nominal=floor + pd.Timedelta(hours=1),
        minutes_from_expected=dev.dt.total_seconds()/60,
        was_offschedule=df['_dt'].notna() & (dev > tol)
    )

    quarantine = df[df['was_offschedule'] | df['_dt'].isna()].copy()
    elig = df[df['within_tol'] & df['_dt'].notna()]
    dup_sizes = elig['dt_nominal'].value_counts()

    chosen = (elig.assign(_dev=dev)
                 .sort_values(['dt_nominal', '_dev', '_dt'])
                 .drop_duplicates('dt_nominal')
                 .assign(dup_count_before=lambda x: x['dt_nominal'].map(dup_sizes).fillna(0).astype(int),
                         was_snapped=True))

    spine = pd.date_range(df['_dt'].min().round('h'), df['_dt'].max().round('h'), freq='h')
    aligned = (chosen.set_index('dt_nominal')
                    .reindex(spine)
                    .rename_axis('t_utc')
                    .assign(was_snapped=lambda x: x['was_snapped'].astype(bool).fillna(False),
                            was_offschedule=lambda x: x['was_offschedule'].astype(bool).fillna(False),
                            dup_count_before=lambda x: x['dup_count_before'].fillna(0).astype(int)))

    hod = aligned.index.hour + aligned.index.minute/60
    doy = aligned.index.dayofyear + hod/24
    aligned = aligned.assign(hod_sin=np.sin(2*np.pi*hod/24),  hod_cos=np.cos(2*np.pi*hod/24),
                             doy_sin=np.sin(2*np.pi*doy/365), doy_cos=np.cos(2*np.pi*doy/365))

    row_count, index_hours = len(chosen), len(spine)
    qa = {
        'rows_total'    : len(df),
        'unparsable'    : int(df['_dt'].isna().sum()),
        'offschedule'   : int(df['was_offschedule'].sum()),
        'duplicates'    : int((dup_sizes > 1).sum()),
        'index_hours'   : index_hours,
        'missing_hours' : max(index_hours - row_count, 0),
        'coverage_pct'  : (len(df) / index_hours),
    }
    return aligned, qa, quarantine

result = dt_treatment(dfr)
df = result[0]
print("*-"*12 + " - QA REPORT - " + "*-"*12)
print(f"start : {df.index.min()} | end : {df.index.max()}")
print(result[1])
print("*-"*32)
df.head(25)

*-*-*-*-*-*-*-*-*-*-*-*- - QA REPORT - *-*-*-*-*-*-*-*-*-*-*-*-
start : 2015-09-27 01:00:00 | end : 2025-08-27 04:00:00
{'rows_total': 86813, 'unparsable': 0, 'offschedule': 4, 'duplicates': 4, 'index_hours': 86932, 'missing_hours': 127, 'coverage_pct': 0.9986311139741407}
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


,STATION,NAME,DATE,SOURCE,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,AA1,AA2,AA3,...,within_tol,minutes_from_expected,was_offschedule,_dev,dup_count_before,was_snapped,hod_sin,hod_cos,doy_sin,doy_cos
t_utc,,,,,,,,,,,,,,,,,,,,,
2015-09-27 01:00:00,7.220700e+10,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T00:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,True,0.0,False,0 days,1,True,2.588190e-01,9.659258e-01,-0.997963,-0.063793
2015-09-27 02:00:00,7.220700e+10,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T01:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,True,0.0,False,0 days,1,True,5.000000e-01,8.660254e-01,-0.998009,-0.063077
2015-09-27 03:00:00,7.220700e+10,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T02:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,True,0.0,False,0 days,1,True,7.071068e-01,7.071068e-01,-0.998054,-0.062361
2015-09-27 04:00:00,7.220700e+10,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T03:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,True,0.0,False,0 days,1,True,8.660254e-01,5.000000e-01,-0.998098,-0.061645
2015-09-27 05:00:00,7.220700e+10,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T04:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,True,0.0,False,0 days,1,True,9.659258e-01,2.588190e-01,-0.998142,-0.060929
2015-09-27 06:00:00,7.220700e+10,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T05:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,True,0.0,False,0 days,1,True,1.000000e+00,6.123234e-17,-0.998186,-0.060213
2015-09-27 07:00:00,7.220700e+10,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T06:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,True,0.0,False,0 days,1,True,9.659258e-01,-2.588190e-01,-0.998228,-0.059497
2015-09-27 08:00:00,7.220700e+10,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T07:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,True,0.0,False,0 days,1,True,8.660254e-01,-5.000000e-01,-0.998271,-0.058781
2015-09-27 09:00:00,7.220700e+10,"SAVANNAH INTERNATIONAL AIRPORT, GA US",2015-09-27T08:53:00,7,FM-15,KSAV,V030,"01,0000,9,5",NaN,NaN,...,True,0.0,False,0 days,1,True,7.071068e-01,-7.071068e-01,-0.998313,-0.058065


In [77]:
nans = df[df['TMP'].isna()]
print(len(nans))
nans

127


,STATION,NAME,DATE,SOURCE,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,AA1,AA2,AA3,...,WA1,WD1,WG1,WJ1,WND,DATE_raw,minutes_from_expected,was_offschedule,dup_count_before,was_snapped
t_utc,,,,,,,,,,,,,,,,,,,,,
2015-12-04 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,0,True
2015-12-09 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,0,True
2015-12-10 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,0,True
2015-12-16 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,0,True
2016-02-03 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,0,True
2025-05-30 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,0,True
2025-05-30 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,0,True


In [11]:
outliers_domain['temp_C'].value_counts()

temp_C
999.9    80
Name: count, dtype: int64

168


,time_est,sin_doy,cos_doy,sin_hod,cos_hod,temp_C,temp_C_flag,C_temp_C
1641,2015-12-04 04:53:00,NaN,NaN,NaN,NaN,8.900000,False,8.900000
1769,2015-12-09 12:53:00,NaN,NaN,NaN,NaN,20.850000,False,20.850000
1775,2015-12-09 18:53:00,NaN,NaN,NaN,NaN,15.250000,False,15.250000
1925,2015-12-16 00:53:00,NaN,NaN,NaN,NaN,10.550000,False,10.550000
3113,2016-02-03 12:53:00,NaN,NaN,NaN,NaN,26.400000,False,26.400000
...,...,...,...,...,...,...,...,...
84107,2025-05-01 06:53:00,NaN,NaN,NaN,NaN,18.600000,False,18.600000
84804,2025-05-30 07:53:00,NaN,NaN,NaN,NaN,23.166667,False,23.166667
84805,2025-05-30 08:53:00,NaN,NaN,NaN,NaN,NaN,False,NaN
85874,2025-07-13 21:53:00,NaN,NaN,NaN,NaN,26.650000,False,26.650000
